In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_json("../input/train.json")
test = pd.read_json("../input/test.json")

In [ ]:
train_test = [train,test]

어떤 방이 인기가 있는지 예측하기 = interest_level 이 어떤 값인지 <br>

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
test.info()

<h3>1. interest_level </h3>

In [ ]:
train['interest_level'].value_counts()

low 가 가장 많고 high가 가장 적음

<h3> 2. bathroom </h3>

In [ ]:
facet = sns.FacetGrid(train, hue = "interest_level", aspect=4)
facet.map(sns.kdeplot, 'bathrooms', shade=True)
facet.add_legend()
plt.show()

분포를 보면 1일때가 가장 많음을 알 수 있고,<br>
bathroom이 1~2 사이일때 가장 high 가 많음을 알 수 있다. <br>
좀 더 자세히 보기 위해 x축을 조절해 보겠다.

In [ ]:
facet = sns.FacetGrid(train, hue = "interest_level", aspect=4)
facet.map(sns.kdeplot, 'bathrooms', shade=True)
facet.set(xlim=(0,2))
facet.add_legend()
plt.show()

In [ ]:
facet = sns.FacetGrid(train, hue = "interest_level", aspect=4)
facet.map(sns.kdeplot, 'bathrooms', shade=True)
facet.set(xlim=(2,6))
facet.add_legend()
plt.show()

In [ ]:
for dataset in train_test:
    dataset.loc[ dataset['bathrooms'] <= 2, 'bathrooms'] = 2,
    dataset.loc[(dataset['bathrooms'] > 2) & (dataset['bathrooms'] <= 4), 'bathrooms'] = 1,
    dataset.loc[ dataset['bathrooms'] > 4, 'bathrooms'] = 0


<h3> 3. bedrooms </h3>

In [ ]:
facet = sns.FacetGrid(train, hue = "interest_level", aspect=4)
facet.map(sns.kdeplot, 'bedrooms', shade=True)
facet.add_legend()
plt.show()

bathrooms 과는 다르게 interest_level에 매우 큰 영향을 주지는 않음<br>
bedroom 이 1인 경우 low의 비율이 매우 높다는 것 정도가 주목할 만 함

In [ ]:
for dataset in train_test:
    dataset.loc[ dataset['bedrooms'] <= 2, 'bedrooms'] = 0,
    dataset.loc[(dataset['bedrooms'] > 2) & (dataset['bedrooms'] <= 4), 'bedrooms'] = 1,
    dataset.loc[ dataset['bedrooms'] > 4, 'bedrooms'] = 2

<h3>4. building_id</h3>

In [ ]:
sum(train['building_id']=='0')

id가 0인것이 8286 개인데, 결측치로 봐도 무방할듯 함 <br>
하지만 building_id 를 어떻게 처리할지 아직 감이 잘 안오므로 넘어가겠다 <br>

<h3>5. created </h3>

In [ ]:
train["created"] = pd.to_datetime(train["created"])
train["month_created"] = train["created"].dt.month

In [ ]:
train["month_created"]

In [ ]:
train['month_created'].value_counts()

In [ ]:
def bar_chart(feature):
    low = train[train['interest_level']=='low'][feature].value_counts() # survived 라는 값에 대해 수를 세줌
    medium = train[train['interest_level']=='medium'][feature].value_counts()
    high = train[train['interest_level']=='high'][feature].value_counts()
    df = pd.DataFrame([low, medium, high])
    df.index = ['low','medium','high']
    df.plot(kind='bar',stacked=True, figsize=(10,5))

In [ ]:
bar_chart('month_created')

month 는 거의 동일한 분포를 가지는듯 함, date로 다시 비교해보자

In [ ]:
train["created"] = pd.to_datetime(train["created"])
train["date_created"] = train["created"].dt.date
cnt_srs = train['date_created'].value_counts()

plt.figure(figsize=(12,4))
ax = plt.subplot(111)
ax.bar(cnt_srs.index, cnt_srs.values, alpha=0.8)
ax.xaxis_date()
plt.xticks(rotation='vertical')
plt.show()

무슨 의미가 있는건지 잘 모르겠음....

In [ ]:
train['day_of_week'] = train['created'].dt.weekday

In [ ]:
test["created"] = pd.to_datetime(test["created"])
test['day_of_week'] = test['created'].dt.weekday

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = sns.countplot(x="day_of_week", hue="interest_level",
                   hue_order=['low', 'medium', 'high'], data=train,
                   order=['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']);
plt.xlabel('Day of Week');
plt.ylabel('Number of occurrences');

### Adding percents over bars
height = [p.get_height() for p in ax.patches]
ncol = int(len(height)/3)
total = [height[i] + height[i + ncol] + height[i + 2*ncol] for i in range(ncol)] * 3
for i, p in enumerate(ax.patches):    
    ax.text(p.get_x()+p.get_width()/2,
            height[i] + 50,
            '{:1.0%}'.format(height[i]/total[i]),
            ha="center") 

In [ ]:
train['created_day'] = train['created'].dt.day

In [ ]:
test['created_day'] = test['created'].dt.day

In [ ]:
### Iterest per Day of Week
fig = plt.figure(figsize=(12,6))
sns.countplot(x="created_day", hue="interest_level", hue_order=['low', 'medium', 'high'], data=train);
plt.xlabel('created_day');
plt.ylabel('Number of occurrences');

<h3>6. features<h3>

In [ ]:
train["num_features"] = train["features"].apply(len)
test["num_features"] = test["features"].apply(len)

뭐하는 건지 잘 모르겠어서... 일단 그냥 수만 세서 전처리를 해놓음

<h3>7. latitude, longitude <h3>

In [ ]:
llimit = np.percentile(train.latitude.values, 1)
ulimit = np.percentile(train.latitude.values, 99)
train['latitude'].ix[train['latitude']<llimit] = llimit
train['latitude'].ix[train['latitude']>ulimit] = ulimit

plt.figure(figsize=(8,6))
sns.distplot(train.latitude.values, bins=50, kde=False)
plt.xlabel('latitude', fontsize=12)
plt.show()

In [ ]:
llimit = np.percentile(train.longitude.values, 1)
ulimit = np.percentile(train.longitude.values, 99)
train['longitude'].ix[train['longitude']<llimit] = llimit
train['longitude'].ix[train['longitude']>ulimit] = ulimit

plt.figure(figsize=(8,6))
sns.distplot(train.longitude.values, bins=50, kde=False)
plt.xlabel('longitude', fontsize=12)
plt.show()

<h3>8. price</h3>
manager_id, listing_id, photos, street_address 는 그냥 버림

In [ ]:
train['price']

In [ ]:
facet = sns.FacetGrid(train, hue = "interest_level", aspect=4)
facet.map(sns.kdeplot, 'price', shade=True)
facet.add_legend()
plt.show()

당연한 것이겠지만, 가격이 높아지면 interest_level 이 거의 low임<br>
x축 범위를 바꾸어 좀 더 살펴보겠음

In [ ]:
facet = sns.FacetGrid(train, hue = "interest_level", aspect=4)
facet.map(sns.kdeplot, 'price', shade=True)
facet.set(xlim=(0,100000))
facet.add_legend()
plt.show()

In [ ]:
facet = sns.FacetGrid(train, hue = "interest_level", aspect=4)
facet.map(sns.kdeplot, 'price', shade=True)
facet.set(xlim=(100000,200000))
facet.add_legend()
plt.show()

In [ ]:
facet = sns.FacetGrid(train, hue = "interest_level", aspect=4)
facet.map(sns.kdeplot, 'price', shade=True)
facet.set(xlim=(0,10000))
facet.add_legend()
plt.show()

0~10000까지는 high 와 medium 이 많음<br>
일단은 범주화 시키지 않고, 결과를 보고 범주화를 시키도록 하겠음 <br>


In [ ]:
train.info()

In [ ]:
features_drop = ['building_id', 'created', 'description', 'display_address', 'features', 'manager_id', 'photos', 'street_address', 'month_created', 'date_created']
train1 = train.drop(features_drop, axis=1)

In [ ]:
features_drop = ['building_id', 'created', 'description', 'display_address', 'features', 'manager_id', 'photos', 'street_address']
test1 = test.drop(features_drop, axis=1)

In [ ]:
X = train[['bathrooms','bedrooms','latitude','longitude','price','day_of_week','created_day','num_features']]

y = train1['interest_level']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

In [ ]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)
log_loss(y_val, y_val_pred)

In [ ]:
X = test[['bathrooms','bedrooms','latitude','longitude','price','day_of_week','created_day','num_features']]

y = clf.predict_proba(X)

In [ ]:
labels2idx = {label: i for i, label in enumerate(clf.classes_)}
labels2idx

In [ ]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y[:, labels2idx[label]]
sub.to_csv("submission_rf.csv", index=False)